# Caratteristiche del notebook

- System_prompt presente = No
- Dataset usato = MALLS_V0
- Epoche = 3
- Fine Tuning eseguito = Si - Repo -> francescoocurcio/new_llama3.2-3B-log-ftn-malls-3epoch_10k-sysprompt_no
- Addestrato solo sulle risposte = No

# Importazione delle librerie e definizione delle costanti

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [ ]:
#########################
# IMPORT DELLE LIBRERIE
#########################
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json
import torch
import pandas as pd
pd.set_option('display.max_columns', None)  # Mostra tutte le colonne
pd.set_option('display.width', None)        # Non tronca l'output a una larghezza fissa
pd.set_option('display.max_colwidth', None)

from datasets import load_dataset
from IPython.display import display
from unsloth import FastLanguageModel, to_sharegpt
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

#########################
# COSTANTI
#########################

MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True

OUTPUT_REPO = "francescoocurcio/new_llama3.2-3B-log-ftn-malls-3epoch_10k-sysprompt_no"
SAVE_DIRECTORY = "/kaggle/working/new_llama3.2-3B-log-ftn-malls-3epoch_10k-sysprompt_no"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-10 13:40:53.945094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746884454.369512      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746884454.484903      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# HuggingFace Login

In [ ]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `UNIVERSAL_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `UNIVERSAL_TOKEN`


# Selezione e configurazione del modello: Llama3.2 3B Instruct

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT
    # token = "hf..." #Use one if using gated models like meta-llama/Llama....
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

#PEFT = Parameter Efficient Fine Tuning
model = FastLanguageModel.get_peft_model( #Modello quantizzato
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Dataset building

In [5]:
df = load_dataset("yuan-yang/MALLS-v0", split="train").to_pandas().sample(n=10000, random_state=42).reset_index(drop=True)
display(df)

README.md:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

MALLS-v0.1-train.json:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

MALLS-v0.1-test.json:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27284 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

,FOL,NL
0,∀x (WritingInstrument(x) ∧ ThinCylindricalBody(x) ∧ ContainsInk(x) ∧ UsesSmallBall(x) ∧ TransfersInkToPaper(x) → BallpointPen(x)),"A writing instrument that has a thin, cylindrical body, contains ink, and uses a small ball to transfer ink to paper is a ballpoint pen."
1,∀x (Machine(x) ∧ PerformsTasksAutonomously(x) ∧ WithoutHumanIntervention(x) → Robot(x)),A machine that can perform tasks autonomously without human intervention is a robot.
2,"∀x∀y∀z (Farmer(x) ∧ Crop(y) ∧ Livestock(z) ∧ Farm(w) → GrowsAndRaises(x, y, z, w))",A farmer grows crops and raises livestock on a farm.
3,∀x (Engine(x) ∧ BurnsFuelInCombustionChamber(x) ∧ GeneratesMechanicalPower(x) → InternalCombustionEngine(x)),An engine is an internal combustion engine if it generates mechanical power by burning fuel in a combustion chamber.
4,∀x (Airplane(x) → CanFly(x)),"If a vehicle is an airplane, it can fly."
...,...,...
9995,"∀x (Parent(x) ↔ ∃y (Child(y) ∧ HasChild(x, y)))",A person is a parent if they have a child.
9996,∀x (Cat(x) ∧ Hungry(x) → Meows(x)),Cats meow when they are hungry.
9997,"∀x ∀y ∀z (Bicycle(x) → Wheels(x, 2)) ∧ (Tricycle(y) → Wheels(y, 3)) ∧ (Unicycle(z) → Wheels(z, 1))","A bicycle has two wheels, a tricycle has three wheels, and a unicycle has one wheel."
9998,∀x (Bike(x) ∧ (FlatTire(x) ∨ BrokenChain(x)) → NeedsRepair(x)),A bike must have at least one flat tire or a broken chain to be considered in need of repair.


In [6]:
from unsloth import to_sharegpt
from datasets import Dataset

dataset = Dataset.from_pandas(df)

dataset = to_sharegpt(
    dataset,
    merged_prompt = "Convert the following Natural Language statement into a First-Order Logic formula.[[:\n{NL}]]",
    output_column_name = "FOL",
    conversation_extension = 1, #Select more to handle longer conversations
)
print(dataset[0])

Merging columns:   0%|          | 0/10000 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'conversations': [{'from': 'human', 'value': "Convert the following Natural Language statement into a First-Order Logic formula.:\n('A writing instrument that has a thin, cylindrical body, contains ink, and uses a small ball to transfer ink to paper is a ballpoint pen.',)"}, {'from': 'gpt', 'value': '∀x (WritingInstrument(x) ∧ ThinCylindricalBody(x) ∧ ContainsInk(x) ∧ UsesSmallBall(x) ∧ TransfersInkToPaper(x) → BallpointPen(x))'}]}


## Opzione di aggiunta per il system prompt

In [ ]:
#system_message = {
#    "from": "system",
#    "value": (
#        "You are a math expert. You will be given a mathematical problem to solve. "
#        "Your aim is to first provide a step-by-step explanation of the solution "
#        "(integrating the formulae in LaTeX syntax) and then to conclude with a clear and concise final answer."
#    )
#}

# Aggiungilo all'inizio di ogni conversazione
#def add_system_message(example):
#    conversation = example["conversations"]
#    return {
#        "conversations": [system_message] + conversation
#    }

# Applica la funzione a tutto il dataset
#dataset = dataset.map(add_system_message)
#print(dataset[0])

# Costruzione dataset finale conversazionale

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

print(dataset)
print(dataset[0])

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 10000
})
{'conversations': [{'content': "Convert the following Natural Language statement into a First-Order Logic formula.:\n('A writing instrument that has a thin, cylindrical body, contains ink, and uses a small ball to transfer ink to paper is a ballpoint pen.',)", 'role': 'user'}, {'content': '∀x (WritingInstrument(x) ∧ ThinCylindricalBody(x) ∧ ContainsInk(x) ∧ UsesSmallBall(x) ∧ TransfersInkToPaper(x) → BallpointPen(x))', 'role': 'assistant'}]}


In [8]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[0]['text']

Model does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nConvert the following Natural Language statement into a First-Order Logic formula.:\n('A writing instrument that has a thin, cylindrical body, contains ink, and uses a small ball to transfer ink to paper is a ballpoint pen.',)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n∀x (WritingInstrument(x) ∧ ThinCylindricalBody(x) ∧ ContainsInk(x) ∧ UsesSmallBall(x) ∧ TransfersInkToPaper(x) → BallpointPen(x))<|eot_id|>"

In [9]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.padding_side)
#Anche dopo aver cambiato inizialmente queste operazioni il tokenizer a fronte delle
#operazioni eseguite è come se eseguisse una sorta di riformattazione

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.padding_side)

<|finetune_right_pad_id|>
128004
right
<|eot_id|>
128009
left


In [10]:
text = dataset[0]['text']
tokenized = tokenizer(text, return_tensors="pt", return_attention_mask=True)
input_ids = tokenized["input_ids"][0]

decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids)

for i, (token_id, token_str) in enumerate(zip(input_ids, decoded_tokens)):
    print(f"{i:03d} | Token ID: {token_id.item():>6} | Token: {repr(token_str)}")

000 | Token ID: 128000 | Token: '<|begin_of_text|>'
001 | Token ID: 128000 | Token: '<|begin_of_text|>'
002 | Token ID: 128006 | Token: '<|start_header_id|>'
003 | Token ID:   9125 | Token: 'system'
004 | Token ID: 128007 | Token: '<|end_header_id|>'
005 | Token ID:    271 | Token: 'ĊĊ'
006 | Token ID:  38766 | Token: 'Cut'
007 | Token ID:   1303 | Token: 'ting'
008 | Token ID:  33025 | Token: 'ĠKnowledge'
009 | Token ID:   2696 | Token: 'ĠDate'
010 | Token ID:     25 | Token: ':'
011 | Token ID:   6790 | Token: 'ĠDecember'
012 | Token ID:    220 | Token: 'Ġ'
013 | Token ID:   2366 | Token: '202'
014 | Token ID:     18 | Token: '3'
015 | Token ID:    198 | Token: 'Ċ'
016 | Token ID:  15724 | Token: 'Today'
017 | Token ID:   2696 | Token: 'ĠDate'
018 | Token ID:     25 | Token: ':'
019 | Token ID:    220 | Token: 'Ġ'
020 | Token ID:   1627 | Token: '26'
021 | Token ID:   5887 | Token: 'ĠJuly'
022 | Token ID:    220 | Token: 'Ġ'
023 | Token ID:   2366 | Token: '202'
024 | Token ID:     1

# Addestramento del modello tramite LoRA

In [11]:
from transformers import TrainerCallback

class LossLoggerCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            loss = logs["loss"]
            step = state.global_step
            self.train_losses.append((step, loss))
            print(f"📉 Step {step} - Loss: {loss:.4f}")

from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

loss_callback = LossLoggerCallback()

#model.config.use_cache = False

training_args = SFTConfig(
    do_train                    = True,

    dataset_text_field          = "text",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,

    num_train_epochs            = 3,   # Epoche complete
    #max_steps                   = 10,
    
    learning_rate               = 2e-4,
    lr_scheduler_type           = "linear",
    logging_strategy            = 'steps',
    logging_steps               = 20,
    # 💾 Salvataggio
    save_strategy               = 'epoch',
    #save_steps                  = 200,

    warmup_steps                = 40,

    optim                       = "adamw_8bit",
    seed                        = 42,

    fp16                        = not is_bfloat16_supported(),
    bf16                        = is_bfloat16_supported(),

    weight_decay                = 0.01,
    report_to                   = "none",
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    dataset_num_proc   = 2,
    max_seq_length     = MAX_SEQ_LENGTH,
    train_dataset      = dataset,
    args               = training_args,
    data_collator      = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing            = False,
    callbacks          = [loss_callback]
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
print("Avvio addestramento LoRA...")
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 3 | Total steps = 936
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Avvio addestramento LoRA...
Unsloth: Will smartly offload gradients to save VRAM!


KeyboardInterrupt: 

# Salvataggio del modello e visualizzazione della loss di addestramento

In [ ]:
!mkdir $SAVE_DIRECTORY

In [ ]:
# PUSH MODELLO LoRA + TOKENIZER su HUGGING FACE
print("🔄 Caricamento del modello e del tokenizer in corso...")
model.push_to_hub(OUTPUT_REPO, token=HF_UNIVERSAL_TOKEN, private=True)
tokenizer.push_to_hub(OUTPUT_REPO, token=HF_UNIVERSAL_TOKEN, private=True)

print(f"✅ Modello caricato correttamente su: {OUTPUT_REPO}\n")
print(f"✅ Tokenizer caricato correttamente su: {OUTPUT_REPO}\n")

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt


# SALVATAGGIO MODELLO LoRA + TOKENIZER
print("💾 Salvataggio del modello e tokenizer...")
trainer.model.save_pretrained(SAVE_DIRECTORY)
tokenizer.save_pretrained(SAVE_DIRECTORY)
print(f"✅ Modello LoRA salvato in: {SAVE_DIRECTORY}")

#Visualizzazione loss di addestramento
def crate_loss_chart(json_file):
    # Carica il file JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Estrai i dati di interesse
    log_history = data.get("log_history", [])
    steps = [entry["step"] for entry in log_history]
    losses = [entry["loss"] for entry in log_history]
    
    # Crea un DataFrame
    df = pd.DataFrame({"Step": steps, "Loss": losses})
    
    # Crea il grafico a linee
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df, x="Step", y="Loss", marker='o')
    plt.title("Andamento della Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    
    # Rotazione delle etichette e selezione dei ticks
    plt.xticks(rotation=45)
    plt.locator_params(axis='x', nbins=10)  # Mostra solo 10 ticks sull'asse x
    
    plt.grid(True)
    plt.tight_layout()  # Migliora la disposizione degli elementi
    plt.show()

loss_path = "/kaggle/working/trainer_output/checkpoint-936/trainer_state.json"
crate_loss_chart(loss_path)